# TM10007 Assignment template

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [29]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

# Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# fetch data
from worclipo.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
print(type(data))


The number of samples: 115
The number of columns: 494
<class 'pandas.core.frame.DataFrame'>


# setting up the data to be processed

In [30]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
# change lipoma = 0 and liposarcoma = 1 and encode labels
group_names = list(set(data.label))
data.loc[data['label'] == 'lipoma', 'label'] = 0
data.loc[data['label'] == 'liposarcoma', 'label'] = 1
data['label'] = pd.cut(data['label'], bins = 2, labels=group_names)
print(data['label'].unique())
label_diag = LabelEncoder()
data['label'] = label_diag.fit_transform(data['label'])

# assign X to measurements and y to outcome (lipoma/sarcoma)
X = data.drop('label', axis=1)
y = data['label']

# splitting the data into test and train
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=10)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=10)

# scale the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_train)


['liposarcoma', 'lipoma']
Categories (2, object): ['lipoma' < 'liposarcoma']
[[-0.06896609  0.19614026 -0.85233803 ... -0.65606844 -1.42908577
  -1.34198741]
 [-1.00021849  0.64282965  0.97477001 ...  1.13432028  0.62755147
   1.17725266]
 [ 0.81313167 -0.57854378  2.679078   ...  0.64115649 -1.42908577
   1.60914239]
 ...
 [-0.84025635  1.55569853 -0.99153506 ... -0.52358715  0.67204122
  -0.90344033]
 [-1.53550315  0.82693849  0.15842472 ... -0.33635456 -1.41952079
   0.32485038]
 [ 0.11284454  0.15136892 -0.83247575 ... -0.45424561  1.21700867
  -0.48646213]]
rfc               precision    recall  f1-score   support

           0       0.75      0.69      0.72        13
           1       0.64      0.70      0.67        10

    accuracy                           0.70        23
   macro avg       0.69      0.70      0.69        23
weighted avg       0.70      0.70      0.70        23

[[9 4]
 [3 7]]


# Classify

In [ ]:

# try a classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

print('rfc', classification_report(y_test, pred_rfc))
print(confusion_matrix(y_test, pred_rfc))